In [17]:
import shutil
import pandas as pd
import sys
import subprocess
import os
import asyncio
from concurrent.futures.thread import ThreadPoolExecutor
import numpy as np
import datetime

In [37]:
og_wd = os.getcwd()
og_wd = str(og_wd).replace(os.sep, '/')
data_folder = og_wd + '/Data/'
template_osw = og_wd + "/template.osw"
sce_pumas = pd.read_csv(data_file_path + "SCE_PUMA_2010.csv")
md = pd.read_parquet(data_folder + "metadata.parquet")

pumas = sce_pumas['GEOID_NREL']
# pumas = ['G06003711']

In [38]:
md = md.loc[md['in.puma'].isin(pumas)]


In [49]:
total_models = len(md)
print(total_models)
sample = len(md.loc[md['in.puma']=='G06003711'])
(total_models/sample)/(24*6)

27327


0.7809499314128944

In [47]:
48/8

6.0

In [16]:
for each_puma in pumas:
    # geoid_nrel = 'G{0:02d}{1:06d}'.format(state, each_puma)
    # geoid_nasa = '{0:01d}{1:05d}'.format(state, each_puma)
      
    
    puma_path = data_folder + each_puma + '/'
    md = pd.read_csv(puma_path + 'Models_Metadata_' + each_puma +'.csv')
#     print(md['in.weather_file_TMY3'])
#     print()
    full_weather =str(md['in.weather_file_2018'].unique())

    # Find the index of the starting position of the substring
    end_index = full_weather.find('_2018.epw')

    # Extract the substring using slicing
    extracted_value = full_weather[end_index - 6:end_index]

    print(extracted_value)

#     https://energyplus-weather.s3.amazonaws.com/north_and_central_america_wmo_region_4/USA/CA/USA_CA_Lancaster-Gen.Wm.Fox.Field.723816_TMY3/USA_CA_Lancaster-Gen.Wm.Fox.Field.723816_TMY3.epw

723816


In [ ]:
import json
import re
from urllib.request import Request, urlopen

path_to_save = '' # create a directory and write the name of directory here
response = urlopen('https://github.com/NREL/EnergyPlus/raw/develop/weather/master.geojson')
data = json.loads(response.read().decode('utf8'))
count = 0
for location in data['features']:
    for file_type in ['epw']:
        match = re.search(r'href=[\'"]?([^\'" >]+)', location['properties'][file_type])
        if match:
            url = match.group(1)
            name = url[url.rfind('/') + 1:]
            count += 1
            print(count, ':', name, '\t')
            response = Request(url, headers={'User-Agent' : "Magic Browser"}) 
            with open(path_to_save + name, 'wb') as f:
                f.write(urlopen(response).read())
print('done!')

In [14]:
for each_puma in pumas:
    puma_path = outputs_path + each_puma
    models =  [folder for folder in os.listdir(puma_path)]
    
    # Make new summary folders
    summary_path = puma_path + "/" + "summary"
    if not os.path.exists(summary_path):
        os.makedirs(summary_path)
        
    # - Electricity
    electricity =pd.DataFrame(columns = models)

    # - Gas 
    natural_gas = pd.DataFrame(columns = models)
    
    for each_model in models:
        model_path = puma_path + '/' + each_model
        if os.path.exists(model_path + '/eplusout.csv'):
            # Read the baseline output file
            data = pd.read_csv(model_path + '/eplusout.csv')
    #         data = data.loc[data['LIVING ZONE:Zone Air Temperature [C](Hourly)'].notnull()]
    #         baseline['Date/Time'] = data['Date/Time']
    #         baseline['Outdoor Temp'] = data['LIVING ZONE:Zone Outdoor Air Drybulb Temperature [C](Hourly)']
    #         baseline['RH'] = data['LIVING ZONE:Zone Air Relative Humidity [%](Hourly)']
    #         baseline['Indoor Temp'] = data['LIVING ZONE:Zone Air Temperature [C](Hourly)']
            electricity[each_model] = data['Electricity:Facility [J](Hourly)']

            if 'NaturalGas:Facility [J](TimeStep)' in data:
                natural_gas[each_model]= data['NaturalGas:Facility [J](TimeStep)']
        else:
            print(each_model + ' Does Not Have Eplusout.csv')
        

bldg0010809 Does Not Have Eplusout.csv
bldg0012911 Does Not Have Eplusout.csv
bldg0019265 Does Not Have Eplusout.csv
bldg0019419 Does Not Have Eplusout.csv
bldg0046000 Does Not Have Eplusout.csv
bldg0052402 Does Not Have Eplusout.csv
bldg0072466 Does Not Have Eplusout.csv
bldg0073901 Does Not Have Eplusout.csv
bldg0082630 Does Not Have Eplusout.csv
bldg0086962 Does Not Have Eplusout.csv
bldg0089660 Does Not Have Eplusout.csv
bldg0096114 Does Not Have Eplusout.csv
bldg0097292 Does Not Have Eplusout.csv
bldg0098915 Does Not Have Eplusout.csv
bldg0103959 Does Not Have Eplusout.csv
bldg0104093 Does Not Have Eplusout.csv
bldg0106894 Does Not Have Eplusout.csv
bldg0109121 Does Not Have Eplusout.csv
bldg0109886 Does Not Have Eplusout.csv
bldg0122614 Does Not Have Eplusout.csv
bldg0133197 Does Not Have Eplusout.csv
bldg0134697 Does Not Have Eplusout.csv
bldg0142235 Does Not Have Eplusout.csv
bldg0144451 Does Not Have Eplusout.csv
bldg0146876 Does Not Have Eplusout.csv
bldg0147975 Does Not Have

In [15]:
electricity

,bldg0001141,bldg0009478,bldg0009697,bldg0016082,bldg0016325,bldg0017645,bldg0017923,bldg0019189,bldg0023616,bldg0026637,...,bldg0532302,bldg0532866,bldg0534485,bldg0534878,bldg0536537,bldg0537124,bldg0537611,bldg0542063,bldg0544763,bldg0545502


In [10]:
natural_gas

,bldg0001616,bldg0007754,bldg0010809,bldg0011433,bldg0012911,bldg0013959,bldg0019265,bldg0019336,bldg0019419,bldg0019431,...,bldg0533984,bldg0536177,bldg0537430,bldg0538847,bldg0542639,bldg0542773,bldg0543977,bldg0544782,bldg0547413,summary
0,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.000000e+00,1.649510e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35035,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35036,1.571884e+06,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35037,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35038,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
24*364

8736